In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer 
import stanza

In [2]:
from nltk import tokenize

In [181]:
test = pd.read_csv('full-review-data.csv')

In [182]:
test

,Unnamed: 0,company,aspect,nb_reviews,neutral,negative,positive,sentiment_scores,neg_example0,neg_example0_margin,...,pos_example5,pos_example5_margin,pos_example6,pos_example6_margin,pos_example7,pos_example7_margin,pos_example8,pos_example8_margin,pos_example9,pos_example9_margin
0,0,booking.com,price,14,0.071151,0.912763,0.016087,"[-0.98880655, -0.94684196, -0.9627897, -0.7852...","<span style=""background-color:rgba(242, 136, ...",75.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,booking.com,delivery,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,booking.com,quality,4,0.132439,0.525038,0.342523,"[-0.9743654, 0.99786687, -0.57104623]","<span style=""background-color:rgba(242, 136, ...",275.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,booking.com,return,3,0.164794,0.829840,0.005366,"[-0.9247967, -0.71440417, -0.834221]","<span style=""background-color:rgba(242, 136, ...",225.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,booking.com,refund,29,0.294731,0.612880,0.092389,"[-0.9690843, -0.9247701, -0.29114228, -0.20675...","<span style=""background-color:rgba(242, 136, ...",25.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,75,debijenkorf.nl,product,7,0.171123,0.538343,0.290535,"[-0.035169434, -0.9530555, -0.95574397, 0.4148...","<span style=""background-color:rgba(242, 136, ...",175.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76,76,debijenkorf.nl,service,34,0.040450,0.557136,0.402414,"[0.9973774, 0.9967442, -0.9846928, -0.31989, -...","<span style=""background-color:rgba(242, 136, ...",75.0,...,"<span style=""background-color:rgba(137, 242, ...",75.0,"<span style=""background-color:rgba(137, 242, ...",75.0,"<span style=""background-color:rgba(137, 242, ...",125.0,"<span style=""background-color:rgba(137, 242, ...",125.0,"<span style=""background-color:rgba(137, 242, ...",200.0
77,77,debijenkorf.nl,customer service,21,0.019116,0.577448,0.403436,"[0.9975756, -0.25251353, -0.9797156, -0.981815...","<span style=""background-color:rgba(242, 136, ...",75.0,...,"<span style=""background-color:rgba(137, 242, ...",175.0,"<span style=""background-color:rgba(137, 242, ...",250.0,NaN,NaN,NaN,NaN,NaN,NaN
78,78,debijenkorf.nl,order,31,0.240824,0.444587,0.314589,"[0.99486977, -0.8313483, 0.045523375, -0.90476...","<span style=""background-color:rgba(242, 136, ...",75.0,...,"<span style=""background-color:rgba(137, 242, ...",50.0,"<span style=""background-color:rgba(137, 242, ...",50.0,"<span style=""background-color:rgba(137, 242, ...",200.0,NaN,NaN,NaN,NaN


In [43]:
test.columns

Index(['Unnamed: 0', 'company', 'aspect', 'nb_reviews', 'neutral', 'negative',
       'positive', 'sentiment_scores', 'neg_example0', 'neg_example0_margin',
       'neg_example1', 'neg_example1_margin', 'neg_example2',
       'neg_example2_margin', 'neg_example3', 'neg_example3_margin',
       'neg_example4', 'neg_example4_margin', 'pos_example0',
       'pos_example0_margin', 'pos_example1', 'pos_example1_margin',
       'pos_example2', 'pos_example2_margin', 'pos_example3',
       'pos_example3_margin', 'pos_example4', 'pos_example4_margin'],
      dtype='object')

In [63]:
test['sentiment_scores']

0     [-0.98880655, -0.94684196, -0.94684196, -0.962...
1                                                   NaN
2                  [-0.9743654, 0.99786687, -0.6418621]
3                  [-0.8756653, -0.71440417, -0.834221]
4     [-0.9690843, -0.9247701, -0.29114228, -0.60961...
                            ...                        
75    [-0.035169434, -0.9530555, -0.96402454, 0.1157...
76    [0.9973774, 0.9967442, -0.9849074, -0.9849074,...
77    [0.9975756, -0.25251353, -0.9797156, -0.981815...
78    [0.99486977, 0.99486977, -0.8313483, -0.831348...
79    [0.99751174, 0.9974402, -0.9801469, 0.90711725...
Name: sentiment_scores, Length: 80, dtype: object

In [65]:
test.head(1)

,Unnamed: 0,company,aspect,nb_reviews,neutral,negative,positive,sentiment_scores,neg_example0,neg_example0_margin,...,pos_example0,pos_example0_margin,pos_example1,pos_example1_margin,pos_example2,pos_example2_margin,pos_example3,pos_example3_margin,pos_example4,pos_example4_margin
0,0,booking.com,price,20,0.073005,0.909658,0.017337,"[-0.98880655, -0.94684196, -0.94684196, -0.962...","<span style=""background-color:rgba(242, 136, ...",75.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
sentiment_scores = [float(i) for i in test['sentiment_scores'][0][1:-1].split(',')]

In [99]:
sentiment_scores

[-0.98880655,
 -0.94684196,
 -0.94684196,
 -0.9627897,
 -0.7852165,
 -0.7852165,
 -0.95812684,
 -0.96908474,
 -0.98474103,
 -0.98474103,
 -0.8894564,
 -0.95695674,
 -0.6427796,
 -0.6427796,
 -0.68381333,
 -0.94465715,
 -0.94465715,
 -0.96829784,
 -0.96829784]

In [180]:
def reviews_categorical_df(sentiment_scores):
    
    scores = []
    
    for score in sentiment_scores:
        if score < -0.3:
            scores.append('negative')
        elif score > 0.3:
            scores.append('positive')
        else:
            scores.append('neutral')
            
    df = pd.DataFrame([{"count": scores.count('negative'), 'sentiment':'negative'},
            {"count": scores.count('neutral'), 'sentiment':'neutral'},
            {"count": scores.count('positive'), 'sentiment':'positive'}])
            
    return df

In [157]:
def reviews_categorical_plot(df):

    fig = px.bar(df, x='sentiment', y='count', color='sentiment',
                 color_discrete_map={'negative': 'rgba(242, 136, 136,1.25)', 
                                     'positive': 'rgba(137, 242, 114,1.25)', 
                                     'neutral':'rgb(131, 133, 132)'})
    
    return fig

In [162]:
tmp = reviews_categorical_df(sentiment_scores)

In [178]:
tmp[tmp['sentiment'] == 'negative']['count'].values[0]

19

In [179]:
tmp[tmp['sentiment'] == 'positive']['count'].values[0]

0

In [177]:
tmp.loc[tmp['sentiment'] == 'positive', 'count'].values[0]

0

In [152]:
tmp = pd.DataFrame(reviews_categorical(sentiment_scores))

In [155]:
import plotly.express as px

fig = px.bar(tmp, x='sentiment', y='count', color='sentiment',
                color_discrete_map={'negative': 'rgba(242, 136, 136,1.25)', 
                                    'positive': 'rgba(137, 242, 114,1.25)', 
                                    'neutral':'rgb(131, 133, 132)'})
fig.show()

In [131]:
import plotly.express as px

long_df = px.data.medals_long()
long_df.head()

,nation,medal,count
0,South Korea,gold,24
1,China,gold,10
2,Canada,gold,9
3,South Korea,silver,13
4,China,silver,15


In [124]:
fig = px.bar(long_df, x="nation", y="count", color="medal", title="Long-Form Input")
fig.show()

In [37]:
combined = pd.read_csv('combined_reviews_eng_CLEAN.csv')

In [35]:
test[test['content'].duplicated()]

,Unnamed: 0,name,nb_reviews,location,rating,header,content,date,responded,response_date,response,company
107,7,Ibrahim,1,NL,5,Good services,Good services,2021-02-10 18:31:31+00:00,0,NaN,-,cheaptickets.nl
125,25,abdalla,1,NL,5,Good,Good service,2020-10-24 11:40:03+00:00,0,NaN,-,cheaptickets.nl
128,28,Kostiantyn,1,NL,5,Good service,Good service,2020-10-03 08:00:37+00:00,0,NaN,-,cheaptickets.nl
320,20,reyhane mokh,4,NL,5,Good service,Good service. No bad experience with them,2021-04-09 14:09:05+00:00,0,NaN,-,coolblue.nl
390,90,Freek Jong,1,NL,5,Super service,Super service,2021-02-27 12:38:10+00:00,0,NaN,-,coolblue.nl


In [17]:
combined2 = pd.read_csv('combined_reviews_eng.csv')
combined = pd.read_csv('combined_reviews_eng_CLEAN.csv')



In [5]:
import aspect_based_sentiment_analysis as absa
recognizer = absa.aux_models.BasicPatternRecognizer()
NLP = absa.load(pattern_recognizer=recognizer)

In [6]:
customer_service_df = combined[combined['content'].str.contains('customer service')]

In [11]:
from tqdm import tqdm

examples = []

for review in tqdm(customer_service_df['content']):
    print(review)
    
    run = NLP(text=review, aspects=['customer service'])
    
    examples.append(run.examples[0])
    break
    


  0%|                                                                                          | 0/166 [00:00<?, ?it/s]

If I could give them a negative 5 star review, I would do it. I cancelled my reservation with the hotel in advance, the hotel sent us a statement stating that the reservation was cancelled. I called booking.com and talked to the customer service agent and manager. They said that they were not refunding my money. According to them, it was the hotel's fault even though it was booking.com that charged my card. I called them so many times to get my money back and they said that there was nothing they could do and that i should be the one dealing with the hotel to get my money back. Never ever book with them or their associate third party websites, pay a little more and book straight with the hotel. This company is awful and disgraceful.


  0%|                                                                                          | 0/166 [00:01<?, ?it/s]


In [13]:
examples[0]

PredictedExample(text="If I could give them a negative 5 star review, I would do it. I cancelled my reservation with the hotel in advance, the hotel sent us a statement stating that the reservation was cancelled. I called booking.com and talked to the customer service agent and manager. They said that they were not refunding my money. According to them, it was the hotel's fault even though it was booking.com that charged my card. I called them so many times to get my money back and they said that there was nothing they could do and that i should be the one dealing with the hotel to get my money back. Never ever book with them or their associate third party websites, pay a little more and book straight with the hotel. This company is awful and disgraceful.", aspect='customer service', sentiment=<Sentiment.negative: 1>, text_tokens=['if', 'i', 'could', 'give', 'them', 'a', 'negative', '5', 'star', 'review', ',', 'i', 'would', 'do', 'it', '.', 'i', 'cancelled', 'my', 'reservation', 'with'

In [14]:
absa.summary(examples[0])

absa.display(examples[0].review)

Sentiment.negative for "customer service"
Scores (neutral/negative/positive): [0.03 0.95 0.02]


### Clean scraped reviews

In [2]:
combined = pd.read_csv('combined_reviews_eng.csv')

combined['content'] = combined.content.str.replace('\n', '. ') # quick little cleaning step 

combined['sentences'] = combined['content'].apply(lambda x: tokenize.sent_tokenize(x))

combined = combined.explode('sentences').reset_index(drop=True)

combined = combined[(combined['sentences'].str.len() > 15)].reset_index(drop=True)

# combined.to_csv('combined_reviews_eng_CLEAN.csv')

### Read cleaned reviews

In [4]:
df = pd.read_csv('combined_reviews_eng_CLEAN.csv')

In [9]:
df = df.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)

In [10]:
df[(df['company'] == 'booking.com') & (
    df['sentences'].str.contains('service'))]

,name,nb_reviews,location,rating,header,content,date,responded,response_date,response,company,sentences
5,David,1,US,1,I cancelled my reservation and booking.com nev...,"If I could give them a negative 5 star review,...",2021-04-20 01:40:14+00:00,0,NaN,-,booking.com,I called booking.com and talked to the custome...
13,Alexander Santos,1,US,1,Terrible experience with this website,Terrible experience with this website. I made ...,2021-04-19 19:26:11+00:00,0,NaN,-,booking.com,"Customer service is awful, I have spent three ..."
66,Roberta,3,GB,1,No customer service…,Had a hotel booking in 2020 which was paid on ...,2021-04-14 15:45:28+00:00,0,NaN,-,booking.com,Awful customer service.
75,Rotem,1,IL,2,Don't book Isrotel hotel rooms on booking.com,I used to use booking.com for hotel reservatio...,2021-04-13 11:17:23+00:00,0,NaN,-,booking.com,They finally agreed to do the change but were ...
99,Daniel Mardasy,5,NL,1,Booked a room through their website,Booked a room through their website. When we w...,2021-04-11 16:48:28+00:00,0,NaN,-,booking.com,We've called booking.com but it took us almost...
102,Daniel Mardasy,5,NL,1,Booked a room through their website,Booked a room through their website. When we w...,2021-04-11 16:48:28+00:00,0,NaN,-,booking.com,An awful experience and an unexpected service ...
139,Jasmin Bhangu,1,GB,1,One year on I am still awaiting a…,One year on I am still awaiting a refund! Alwa...,2021-04-10 08:19:29+00:00,0,NaN,-,booking.com,This is appalling service.
142,Jasmin Bhangu,1,GB,1,One year on I am still awaiting a…,One year on I am still awaiting a refund! Alwa...,2021-04-10 08:19:29+00:00,0,NaN,-,booking.com,I would advise to use another service and keep...
152,Mr Richard Chamberlain,27,GB,1,Not recommended,Not recommended. Customer service does not ex...,2021-04-09 18:26:13+00:00,0,NaN,-,booking.com,Customer service does not exist with this comp...
162,Sebastian Adamczuk,7,PL,1,Shiitt company,Shiitt company. Anything goes wrong and they l...,2021-04-08 17:17:57+00:00,0,NaN,-,booking.com,Booked accomodation via this service and then ...


In [ ]:
from bs4 import BeautifulSoup

In [17]:
companies = df['company'].unique()

In [18]:
aspects = ['price', 'delivery']

In [ ]:
# input: df, company & aspect
# output: Dictionary(columns)

In [ ]:
def run_absa(aspect):

    sentences = []

    for sentence in stqdm(df['sentences']):

        run = nlp(text=sentence, aspects=[aspect])
        sentences.append(run.examples[0])

    return sentences

def retrieve_scores(absa_analysis):

    avg_distribution = np.sum(np.array(
        [example.scores for example in absa_analysis]), axis=0) / len(absa_analysis)

    neutral, negative, positive = avg_distribution

    return neutral, negative, positive

In [ ]:
def absa_iteration(df, company, aspect):
    
    some_dict = {}
    
    #### Inputs
    some_dict['company'] = company
    some_dict['aspect'] = aspect
    
    
    #### Metrics
    nb_reviews = df[(df['company'] == company) & (df['sentences'].str.contains(aspect))]
    some_dict['nb_reviews'] = nb_reviews
    
    # Run abstract based sentiment analysis
    absa_analysis = run_absa(aspect)
    neutral, negative, positive = retrieve_scores(absa_analysis)
    
    some_dict['neutral'] = neutral
    some_dict['negative'] = negative
    some_dict['positive'] = positive
    
    #### Plot input
    sentiment_scores = [review.scores[2] - review.scores[1] for review in absa_analysis]
    some_dict['sentiment_scores'] = sentiment_scores
    
    #### Examples
    
    idx_pos_5 = np.array([sent.scores[2] for sent in absa_analysis]).argsort()[-5:][::-1]
    idx_neg_5 = np.array([sent.scores[1] for sent in absa_analysis]).argsort()[-5:][::-1]
    
    str_to_remove = '<span style="background-color:rgba(180,180,180,1.1111111111111112);">Importance 1.00</span>'
    
    for idx_neg in idx_neg_5:

        neg_example = absa_analysis[idx_neg].scores

        if neg_example[1] > 0.5:

            worst_html = display(absa_analysis[idx_neg].review).replace(
                str_to_remove, '').replace('135,206,250', '242, 136, 136')

            soup = BeautifulSoup(worst_html.split('<br>')[0])

            length = len(' '.join([tag.string for tag in soup.find_all('span')]))

            text, margin = worst_html.split('<br>')[0], 25 + length // 100 * 25

        else:
            break
    
    
    some_dict['pos_examples'] = pos_examples
    
    some_dict['neg_examples'] = neg_examples
    
    return some_dict

In [9]:
import aspect_based_sentiment_analysis as absa
recognizer = absa.aux_models.BasicPatternRecognizer()
NLP = absa.load(pattern_recognizer=recognizer)

In [8]:
customer_service_df = combined[combined['sentences'].str.contains('customer service')]

In [10]:
from tqdm import tqdm

examples = []

for sentence in tqdm(customer_service_df['sentences']):
    
    run = NLP(text=sentence, aspects=['customer service'])
    
    examples.append(run.examples[0])
    


100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [00:09<00:00,  2.52it/s]


In [11]:
import numpy as np

In [12]:
aspect = 'customer service'

# Average distribution (neutral | negative | positive)
avg_distribution = np.sum(np.array([example.scores for example in examples]), axis=0) / len(examples)

print(f'Reviews mentioning {aspect}')
avg_distribution

Reviews mentioning customer service


array([0.05962507, 0.7127558 , 0.22761922], dtype=float32)

In [38]:
sentiment_scores = [example.scores[2] - example.scores[1] for idx, example in enumerate(examples)]

In [82]:
sentiment_scores

[0.99726415,
 0.99751806,
 -0.9805648,
 -0.71340114,
 -0.9855761,
 -0.9879231,
 0.9974816,
 0.99731296,
 -0.9870893,
 -0.9166326,
 -0.5877112,
 0.9974704,
 -0.98729986,
 -0.15822676,
 -0.9850764,
 -0.9873056,
 -0.6892363,
 -0.6380442,
 -0.9784847,
 -0.96192706,
 -0.95916384,
 -0.96119434,
 -0.9751521,
 -0.6888849,
 -0.98656595]

In [98]:
fig = px.histogram(sentiment_scores, nbins=25)

fig.update_layout(
    xaxis_title_text = 'Sentiment',
    yaxis_title_text = 'Count',
    bargap=0.2,
    showlegend=False,
    dragmode=False,
    clickmode='none',
)


# fig.update_traces(hovertemplate=None)
fig.update_traces(hovertemplate=
                  '<i><b>Count</i>: %{y}' + 
                  '<br><b>Sentiment Range</b>: %{x}' +
                  '<br><extra></extra>')



fig.show()

In [87]:
sentiment_scores

[0.99726415,
 0.99751806,
 -0.9805648,
 -0.71340114,
 -0.9855761,
 -0.9879231,
 0.9974816,
 0.99731296,
 -0.9870893,
 -0.9166326,
 -0.5877112,
 0.9974704,
 -0.98729986,
 -0.15822676,
 -0.9850764,
 -0.9873056,
 -0.6892363,
 -0.6380442,
 -0.9784847,
 -0.96192706,
 -0.95916384,
 -0.96119434,
 -0.9751521,
 -0.6888849,
 -0.98656595]

In [15]:
pos_scores.argsort()[-5:][::-1]

array([11,  6,  1,  7,  0], dtype=int64)

In [14]:
# positive scores
pos_scores = np.array([example.scores[2] for idx, example in enumerate(examples)])
pos_scores

array([0.99834883, 0.99845755, 0.0055188 , 0.10376485, 0.00646426,
       0.00528512, 0.99850106, 0.9984359 , 0.00569052, 0.01087566,
       0.14215325, 0.99853694, 0.00552637, 0.04802295, 0.00426602,
       0.005228  , 0.14547388, 0.04720908, 0.01017662, 0.00662997,
       0.00800654, 0.01455655, 0.0069249 , 0.11073714, 0.00568936],
      dtype=float32)

In [24]:
best_ordered = np.array([example.scores[2] for idx, example in enumerate(examples)]).argsort()[::-1]

In [28]:
[score for score in pos_scores[best_ordered] if score > 0.6][:5]

[0.99853694, 0.99850106, 0.99845755, 0.9984359, 0.99834883]

In [23]:
pos_scores

array([0.99834883, 0.99845755, 0.0055188 , 0.10376485, 0.00646426,
       0.00528512, 0.99850106, 0.9984359 , 0.00569052, 0.01087566,
       0.14215325, 0.99853694, 0.00552637, 0.04802295, 0.00426602,
       0.005228  , 0.14547388, 0.04720908, 0.01017662, 0.00662997,
       0.00800654, 0.01455655, 0.0069249 , 0.11073714, 0.00568936],
      dtype=float32)

In [19]:
# negative scores
np.array([example.scores[1] for idx, example in enumerate(examples)])

array([1.0846786e-03, 9.3948544e-04, 9.8608357e-01, 8.1716597e-01,
       9.9204034e-01, 9.9320823e-01, 1.0194661e-03, 1.1229619e-03,
       9.9277979e-01, 9.2750823e-01, 7.2986448e-01, 1.0665880e-03,
       9.9282622e-01, 2.0624970e-01, 9.8934245e-01, 9.9253356e-01,
       8.3471018e-01, 6.8525326e-01, 9.8866129e-01, 9.6855700e-01,
       9.6717036e-01, 9.7575086e-01, 9.8207700e-01, 7.9962206e-01,
       9.9225533e-01], dtype=float32)

In [17]:
[example.scores for idx, example in enumerate(examples)]

[[0.0005665521, 0.0010846786, 0.99834883],
 [0.0006029623, 0.00093948544, 0.99845755],
 [0.008397679, 0.98608357, 0.005518797],
 [0.0790692, 0.817166, 0.10376485],
 [0.0014954025, 0.99204034, 0.0064642555],
 [0.0015066285, 0.9932082, 0.0052851154],
 [0.00047944364, 0.0010194661, 0.99850106],
 [0.0004410581, 0.0011229619, 0.9984359],
 [0.0015296724, 0.9927798, 0.005690519],
 [0.061616074, 0.92750823, 0.01087566],
 [0.12798229, 0.7298645, 0.14215325],
 [0.00039649796, 0.001066588, 0.99853694],
 [0.0016474467, 0.9928262, 0.0055263657],
 [0.74572736, 0.2062497, 0.04802295],
 [0.0063915257, 0.98934245, 0.0042660222],
 [0.0022383155, 0.99253356, 0.005228005],
 [0.019815892, 0.8347102, 0.14547388],
 [0.26753768, 0.68525326, 0.047209077],
 [0.0011621026, 0.9886613, 0.010176622],
 [0.024813004, 0.968557, 0.0066299695],
 [0.024823045, 0.96717036, 0.008006539],
 [0.009692586, 0.97575086, 0.014556548],
 [0.010998204, 0.982077, 0.006924901],
 [0.08964077, 0.79962206, 0.11073714],
 [0.0020553458, 0.

In [46]:
examples[0].scores

[0.0005665521, 0.0010846786, 0.99834883]

In [21]:
for idx, _ in enumerate(examples):

    absa.summary(examples[idx])

    absa.display(examples[idx].review)

Sentiment.positive for "customer service"
Scores (neutral/negative/positive): [0.001 0.001 0.998]


Sentiment.positive for "customer service"
Scores (neutral/negative/positive): [0.001 0.001 0.998]


Sentiment.negative for "customer service"
Scores (neutral/negative/positive): [0.008 0.986 0.006]


Sentiment.negative for "customer service"
Scores (neutral/negative/positive): [0.079 0.817 0.104]


Sentiment.negative for "customer service"
Scores (neutral/negative/positive): [0.001 0.992 0.006]


...

Sentiment.negative for "customer service"
Scores (neutral/negative/positive): [0.025 0.967 0.008]


Sentiment.negative for "customer service"
Scores (neutral/negative/positive): [0.01  0.976 0.015]


Sentiment.negative for "customer service"
Scores (neutral/negative/positive): [0.011 0.982 0.007]


Sentiment.negative for "customer service"
Scores (neutral/negative/positive): [0.09  0.8   0.111]


Sentiment.negative for "customer service"
Scores (neutral/negative/positive): [0.002 0.992 0.006]


### FIRST EXAMPLE

In [35]:
import aspect_based_sentiment_analysis as absa

recognizer = absa.aux_models.BasicPatternRecognizer()
NLP = absa.load(pattern_recognizer=recognizer)

print('Text to execute ABSA on:')
print(bol['content'][4])

test_run = NLP(text=bol['content'][4], aspects=['customer', 'price', 'delivery'])

customer, price, delivery = test_run.examples

In [36]:
absa.summary(customer)
absa.display(customer.review)

Sentiment.positive for "customer"
Scores (neutral/negative/positive): [0.001 0.001 0.998]


In [28]:
absa.summary(price)
absa.display(price.review)

Sentiment.positive for "price"
Scores (neutral/negative/positive): [0.162 0.336 0.502]


In [29]:
absa.summary(delivery)
absa.display(delivery.review)

Sentiment.negative for "delivery"
Scores (neutral/negative/positive): [0.161 0.625 0.214]


### OLD CODE

In [9]:
# Make sure you have downloaded the StanfordNLP English model and other essential tools using,
# stanfordnlp.download('en')
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')

def aspect_sentiment_analysis(txt, stop_words, nlp):
    
    txt = txt.lower() # LowerCasing the given Text
    sentList = nltk.sent_tokenize(txt) # Splitting the text into sentences

    fcluster = []
    totalfeatureList = []
    finalcluster = []
    dic = {}

    for line in sentList:
        newtaggedList = []
        txt_list = nltk.word_tokenize(line) # Splitting up into words
        taggedList = nltk.pos_tag(txt_list) # Doing Part-of-Speech Tagging to each word

        newwordList = []
        flag = 0
        for i in range(0,len(taggedList)-1):
            if(taggedList[i][1]=="NN" and taggedList[i+1][1]=="NN"): # If two consecutive words are Nouns then they are joined together
                newwordList.append(taggedList[i][0]+taggedList[i+1][0])
                flag=1
            else:
                if(flag==1):
                    flag=0
                    continue
                newwordList.append(taggedList[i][0])
                if(i==len(taggedList)-2):
                    newwordList.append(taggedList[i+1][0])

        finaltxt = ' '.join(word for word in newwordList) 
        new_txt_list = nltk.word_tokenize(finaltxt)
        wordsList = [w for w in new_txt_list if not w in stop_words]
        taggedList = nltk.pos_tag(wordsList)

        doc = nlp(finaltxt) # Object of Stanford NLP Pipeleine
        
        # Getting the dependency relations betwwen the words
        dep_node = []

        for dep_edge in doc.sentences[0].dependencies:
            dep_node.append([dep_edge[2].text, dep_edge[0].id, dep_edge[1]])

        # Coverting it into appropriate format
        for i in range(0, len(dep_node)):
            if (int(dep_node[i][1]) != 0):
                dep_node[i][1] = newwordList[(int(dep_node[i][1]) - 1)]

        featureList = []
        categories = []
        for i in taggedList:
            if(i[1]=='JJ' or i[1]=='NN' or i[1]=='JJR' or i[1]=='NNS' or i[1]=='RB'):
                featureList.append(list(i)) # For features for each sentence
                totalfeatureList.append(list(i)) # Stores the features of all the sentences in the text
                categories.append(i[0])

        for i in featureList:
            filist = []
            for j in dep_node:
                if((j[0]==i[0] or j[1]==i[0]) and (j[2] in ["nsubj", "acl:relcl", "obj", "dobj", "agent", "advmod", "amod", "neg", "prep_of", "acomp", "xcomp", "compound"])):
                    if(j[0]==i[0]):
                        filist.append(j[1])
                    else:
                        filist.append(j[0])
            fcluster.append([i[0], filist])
            
    for i in totalfeatureList:
        dic[i[0]] = i[1]
    
    for i in fcluster:
        if(dic[i[0]]=="NN"):
            finalcluster.append(i)
        
    return finalcluster

nlp = stanza.Pipeline('en')
stop_words = set(stopwords.words('english'))
# txt = "De muziek kwaliteit is erg goed maar de levensduur van de batterij is verschrikkelijk."
txt = "This is an super interesting testrun that has gone horribly wrong."


test = aspect_sentiment_analysis(txt, stop_words, nlp)

2021-04-11 11:46:35 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| pos       | combined  |
| lemma     | combined  |
| depparse  | combined  |
| sentiment | sstplus   |
| ner       | ontonotes |

2021-04-11 11:46:35 INFO: Use device: gpu
2021-04-11 11:46:35 INFO: Loading: tokenize
2021-04-11 11:46:36 INFO: Loading: pos
2021-04-11 11:46:36 INFO: Loading: lemma
2021-04-11 11:46:37 INFO: Loading: depparse
2021-04-11 11:46:37 INFO: Loading: sentiment
2021-04-11 11:46:37 INFO: Loading: ner
2021-04-11 11:46:38 INFO: Done loading processors!
